In [1]:
import os
import pandas as pd
from gimmemotifs.preprocessing import combine_peaks
import subprocess as sp

Matplotlib is building the font cache; this may take a moment.


In [2]:
#paths to needed files
genome_path_size = '/scratch/bacint/jdeleuw/genomes/hg38/hg38.fa.sizes'

In [3]:
paths = pd.read_csv('/scratch/bacint/jdeleuw/dev_ananse/ananse_paths.csv', sep = ',', header = 0, index_col = 0) 

In [4]:
#Copy narrowPeak files to the ananse map and change the name to the day of differentiation
for narrow_peak, output_dir in zip(paths['narrow_peak'], paths['narrow_peak_output']):
    narrowPeak_df = pd.read_csv(narrow_peak, sep = '\t', header = None)
    narrowPeak_df.to_csv(output_dir, sep = '\t', header = None, index = False)

In [5]:
peak_files = []

for index, cell_type in paths.iterrows():
    peak_files.extend(paths.narrow_peak_output[index].split(' '))

all_peaks_merged = combine_peaks(peak_files,
                                 genome_path_size,
                                 int(200),
                                 True)
#extend each summit to 200bp
all_peaks_merged.start = all_peaks_merged.start.astype(int) - 100
all_peaks_merged.end = all_peaks_merged.end.astype(int) + 100

all_peaks_merged = all_peaks_merged[~all_peaks_merged.chrom.str.contains("GL|Un|KI|MT|X|Y")]

all_peaks_merged.to_csv(
    '/scratch/bacint/jdeleuw/dev_ananse/enhancer_data/peaks_merged.tsv',
    sep="\t",
    header=False,
    index=False,
)

In [7]:
#delete the config file of gimmemotifs
sp.check_call(f'rm -r '
              f'/mbshome/jdeleuw/.config/gimmemotifs/',
              shell = True)

0

In [6]:
#run ananse binding
#specify ATAC bam files
ATAC_bamfiles_duplicate_A = []
ATAC_bamfiles_duplicate_B = []

for index, values in paths.iterrows():
    if index.endswith('A') or index.endswith('KC2'):
        ATAC_bamfiles_duplicate_A.append(values.ATAC_bam)
    else:
        ATAC_bamfiles_duplicate_B.append(values.ATAC_bam)
            
#specify merged peak file
merged_peak = '/scratch/bacint/jdeleuw/ananse/enhancer_data/peaks_merged.tsv'
#specify output paths
binding_paths = []
binding_paths.extend(paths.binding_output.dropna())
#specify log output paths
binding_log = []
binding_log.extend(paths.binding_log.dropna())

for ATAC_bam_A, ATAC_bam_B, output_dir, log in zip(ATAC_bamfiles_duplicate_A, 
                                                   ATAC_bamfiles_duplicate_B, 
                                                   binding_paths, 
                                                   binding_log):
    sp.check_call(f'nice -15 ananse binding '
                  f'-A {ATAC_bam_A} {ATAC_bam_B} '
                  f'-o {output_dir} '
                  f'-r {merged_peak} '
                  f'-g hg38 '
                  f'2> {log}',
                  shell = True)  

In [7]:
#run ananse network
#specify TPM_paths
TPM_duplicate_A = []
TPM_duplicate_B = []

for index, values in paths.iterrows():
    if index.endswith('A') or index.endswith('KC2'):
        TPM_duplicate_A.append(values.TPM_files)
    else:
        TPM_duplicate_B.append(values.TPM_files)
#specify binding input files
binding_input = []
for path in binding_paths:
    path += '/binding.tsv'
    binding_input.append(path)
#specify network output
network_paths = []
network_paths.extend(paths.network_output.dropna())
#specify log output
network_log = []
network_log.extend(paths.network_log.dropna())

for TPM_A, TPM_B, binding_file, output_dir, log in zip(TPM_duplicate_A[1:], 
                                                       TPM_duplicate_B[1:], 
                                                       binding_input[1:], 
                                                       network_paths[1:], 
                                                       network_log[1:]):
    sp.check_call(f'nice -15 ananse network '
                  f'-e {TPM_A} {TPM_B} '
                  f'-b {binding_file} '
                  f'-o {output_dir} '
                  f'-g hg38 '
                  f'2> {log}',
                  shell = True) 

In [ ]:
#run ananse influence
#Specify DEgenes paths
DEgenes_paths = []
DEgenes_paths.extend(paths.DEgenes.dropna())
#Specify influence output
influence_paths = []
influence_paths.extend(paths.influence_output.dropna())
#Specify log output
influence_log = []
influence_log.extend(paths.influence_log.dropna())
for network, DEgenes, output_dir, log in zip(network_paths[1:], 
                                             DEgenes_paths, 
                                             influence_paths, 
                                             influence_log):
    sp.check_call(f'nice -15 ananse influence '
                  f'-s /scratch/bacint/jdeleuw/dev_ananse/network_data/network_Day0.txt '
                  f'-t {network} '
                  f'-d {DEgenes} '
                  f'-p '
                  f'-o {output_dir} ' 
                  f'2> {log}',
                  shell = True) 

In [ ]:
#run ananse influence with day 0 as the target
#Specify DEgenes paths
rev_DEgenes_paths = []
rev_DEgenes_paths.extend(paths.reverse_DEgenes.dropna())
#Specify influence output
rev_influence_paths = []
rev_influence_paths.extend(paths.rev_influence_output.dropna())
#Specify log output
rev_influence_log = []
rev_influence_log.extend(paths.rev_influence_log.dropna())
for network, DEgenes, output_dir, log in zip(network_paths[1:], 
                                             rev_DEgenes_paths, 
                                             rev_influence_paths, 
                                             rev_influence_log):
    sp.check_call(f'nice -15 ananse influence '
                  f'-s {network} '
                  f'-t /scratch/bacint/jdeleuw/dev_ananse/network_data/network_Day0.txt '
                  f'-d {DEgenes} '
                  f'-p '
                  f'-o {output_dir} ' 
                  f'2> {log}',
                  shell = True) 